<a href="https://colab.research.google.com/github/Annmodels/mnist_recon/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

In [0]:
!mkdir MNIST_Fashion
!mv *.gz MNIST_Fashion/

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_Fashion/')

In [0]:
plt.figure(figsize = (1,1))
sample_image = mnist.train.next_batch(1)[0]
sample_image = sample_image.reshape([28,28])
plt.imshow(sample_image , cmap = "gray")
plt.show()

In [0]:
lrate = 0.002
batch_size = 200
epochs = 50000

In [0]:
image_dim = 784
gen_hidn_dim = 400
disc_hidn_dim = 400
z_noise_dim = 100

In [0]:
def xavier_init(shape):
  return tf.random_normal(shape = shape , stddev = 1./tf.sqrt(shape[0]/2.0))

In [0]:
weights = {
    "disc_h": tf.Variable(xavier_init([image_dim , disc_hidn_dim])),
    "disc_final": tf.Variable(xavier_init([disc_hidn_dim , 1])),
    "gen_h": tf.Variable(xavier_init([z_noise_dim , gen_hidn_dim])),
    "gen_final": tf.Variable(xavier_init([gen_hidn_dim , image_dim]))
          }
bias = {
    "disc_h": tf.Variable(xavier_init([disc_hidn_dim])),
    "disc_final": tf.Variable(xavier_init([1])),
    "gen_h": tf.Variable(xavier_init([gen_hidn_dim])),
    "gen_final": tf.Variable(xavier_init([image_dim]))
       }

In [0]:
def Discriminator(x):
  hidn_layer = tf.nn.relu(tf.add(tf.matmul(x , weights["disc_h"]) , bias["disc_h"]))
  final_layer = tf.add(tf.matmul(hidn_layer , weights["disc_final"]) , bias["disc_final"])
  disc_output = tf.nn.sigmoid(final_layer)
  return final_layer,disc_output

def Generator(x):
  hidn_layer = tf.nn.relu(tf.add(tf.matmul(x , weights["gen_h"]) , bias["gen_h"]))
  final_layer = tf.add(tf.matmul(hidn_layer , weights["gen_final"]) , bias["gen_final"])
  gen_output = tf.nn.sigmoid(final_layer)
  return gen_output

In [0]:
z_input = tf.placeholder(tf.float32 , shape = [None , z_noise_dim] , name = "input_noise")
x_input = tf.placeholder(tf.float32 , shape = [None , image_dim] , name = "real_input")

In [0]:
with tf.name_scope("Generator")as scope:
  output_gen = Generator(z_input)

with tf.name_scope("Discriminator")as scope:
  real_output1_disc,real_output_disc = Discriminator(x_input)
  fake_output1_disc,fake_output_disc = Discriminator(output_gen)

with tf.name_scope("Discriminator_Loss")as scope:
  Discriminator_Loss = - tf.reduce_mean(tf.log(real_output_disc + 0.0001)+tf.log(1.-fake_output_disc + 0.0001))

with tf.name_scope("Genetaror_loss")as scope:
  Generator_Loss = -tf.reduce_mean(tf.log(fake_output_disc + 0.0001)) 

In [0]:
Generator_var = [weights["gen_h"],weights["gen_final"],bias["gen_h"],bias["gen_final"]]
Discriminator_var = [weights["disc_h"],weights["disc_final"],bias["disc_h"],bias["disc_final"]]

In [0]:
with tf.name_scope("Optimizer_Discriminator")as scope:
  Discriminator_optimize = tf.train.AdamOptimizer(learning_rate = lrate).minimize(Discriminator_Loss , var_list = Discriminator_var)

with tf.name_scope("Optimizer_Generator")as scope:
  Generator_optimize = tf.train.AdamOptimizer(learning_rate = lrate).minimize(Generator_Loss , var_list = Generator_var)

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  x_batch,_ = mnist.train.next_batch(batch_size)
  z_noise = np.random.uniform(-1.,1.,size = [batch_size , z_noise_dim]) 
  _,Disc_loss_epoch = sess.run([Discriminator_optimize,Discriminator_Loss],feed_dict = {x_input:x_batch,z_input:z_noise}) 
  _,Gen_loss_epoch = sess.run([Generator_optimize,Generator_Loss],feed_dict = {z_input:z_noise})
  if epoch%2000 == 0:
    print("steps: {0}  G_loss: {1} , D_loss: {2}".format(epoch,Gen_loss_epoch,Disc_loss_epoch))

In [0]:
n = 6
canvas = np.empty((28*n,28*n))
for i in range(n):
  z_noise = np.random.uniform(-1.,1.,size = [batch_size , z_noise_dim])
  g = sess.run(output_gen , feed_dict = {z_input:z_noise})
  g = -1*(g-1)
  for j in range(n):
    canvas[i*28:(i+1)*28 , j*28:(j+1)*28] = g[j].reshape([28,28])

plt.figure(figsize = (n,n))
plt.imshow(canvas , origin = "upper" , cmap = "gray")
plt.show()